In [1]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from nuscenes.nuscenes import NuScenes

class NuScenesLidarDataset(Dataset):
    def __init__(self, path, num_classes=32, version='v1.0-mini', lidarseg_path='lidarseg', split="test"):
        self.nusc = NuScenes(version=version, dataroot=path, verbose=False)
        self.lidarseg_path = os.path.join(path, lidarseg_path, version)
        self.num_classes = num_classes

    def __len__(self):
        return len(self.nusc.sample)

    def __getitem__(self, idx):
        sample = self.nusc.sample[idx]
        lidar_token = sample['data']['LIDAR_TOP']
        lidar_data = self.nusc.get('sample_data', lidar_token)
        lidar_filepath = os.path.join(self.nusc.dataroot, lidar_data['filename'])
        lidar_points = np.fromfile(lidar_filepath, dtype=np.float32).reshape(-1, 5)

        # Load corresponding segmentation labels
        lidarseg_filepath = os.path.join(self.lidarseg_path, lidar_token + '_lidarseg.bin')
        if os.path.exists(lidarseg_filepath):
            labels = np.fromfile(lidarseg_filepath, dtype=np.uint8)
        else:
            labels = np.zeros(len(lidar_points), dtype=np.uint8)

        labels = labels[:, np.newaxis]

        return lidar_points, labels, lidar_token


In [2]:
os.makedirs('../data/nuscenes/train', exist_ok=True)

In [3]:
nusc = NuScenesLidarDataset('/data/sets/nuscenes')

In [4]:
max_xyz = np.zeros(3)
min_xyz = np.zeros(3)
for i in range(len(nusc)):
    xyz = nusc[i][0][:, :3]
    center = (xyz.min(axis=0) + xyz.max(axis=0)) / 2.0
    xyz -= center
    max_xyz = np.maximum(max_xyz, xyz.max(axis=0))
    min_xyz = np.minimum(min_xyz, xyz.min(axis=0))
print(max_xyz, min_xyz)

[ 98.83724976 103.82437134  29.0141201 ] [ -98.83724976 -103.82437134  -29.0141201 ]


In [5]:
scale_factor = max(max_xyz - min_xyz)
print(scale_factor)

207.64874267578125


In [6]:
with open('../data/nuscenes/v1.0-mini.txt', 'w+') as f:
	for i in range(len(nusc)):
		lidar_points, labels, scene_token = nusc[i]
		output = {
			'points': lidar_points[:, :3],
			'colors': lidar_points[:, 3:],
			'labels': labels,
		}
		np.savez('../data/nuscenes/train/' + scene_token + '.npz', **output)
		f.write(scene_token + '.npz\n')